In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7'

In [ ]:
%run -p ../train_cnf_disentangle_2cond_nosep.py --data colormnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/cnf_conditional_disentangle_colormnist_bs900_sratio_1_2th_drop_0_5_2cond_linear_nosep_run1 --resume ../experiments_published/cnf_conditional_disentangle_colormnist_bs900_sratio_1_2th_drop_0_5_2cond_linear_nosep_run1/current_checkpt.pth --seed 1 --lr 0.0001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --y_color 10 --y_class 10
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_2cond_nosep.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", 